In [1]:
%env FLASK_ENV=development

env: FLASK_ENV=development


In [2]:
from dotenv import load_dotenv

load_dotenv()

from mongoengine import connect, disconnect

from settings import DevelopmentConfig

from recommender.models import Service, User
from recommender.engines.nlp_embedders.embedders import Services2tensorsEmbedder, Users2tensorsEmbedder

2022-06-18 18:07:49.944642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-06-18 18:07:49.944664: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-06-18 18:07:49.944939: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
Services2tensorsEmbedder()
Users2tensorsEmbedder()

In [4]:
disconnect()
connection = connect(host=DevelopmentConfig.MONGODB_HOST)

### Object to Vector Concept

object -> vec

object has known structure so it's easier than variable structure. We can do one net architecture for given structure :)

Input data models is:
List of dicts of lists of dicts of strings (there is no error in this phrase XD)

### Example of data
```
batch = [
    object1 = {
        "field1": [
            {
                "sub_field1_1": "val1",
                "sub_field1_2": "val2"
            },
            {
                "sub_field1_1": "val3",
                "sub_field1_2": "val4"
            }
        ],
        "field2": [
            {
                "sub_field2_1": "val5",
                "sub_field2_2": "val6"
            },
            {
                "sub_field2_1": "val7",
                "sub_field2_2": "val8"
            }
        ],
    },
    object2 = {
        "field1": [
            {
                "sub_field1_1": "val9",
                "sub_field1_2": "val10"
            },
            {
                "sub_field1_1": "val11",
                "sub_field1_2": "val12"
            }
        ],
        "field2": [
            {
                "sub_field2_1": "val13",
                "sub_field2_2": "val14"
            },
            {
                "sub_field2_1": "val15",
                "sub_field2_2": "val16"
            }
        ],
    },
]
```
### Next step

```
batch = [
    object1 = {
        "sub_field1_1": ["val1", "val3"],
        "sub_field1_2": ["val2", "val4"],
        "sub_field2_1": ["val5", "val7"],
        "sub_field2_2": ["val6", "val8"],
    },
    object2 = {
        "sub_field1_1": ["val9", "val11"],
        "sub_field1_2": ["val10", "val12"],
        "sub_field2_1": ["val13", "val15"],
        "sub_field2_2": ["val14", "val16"],
    },
]
```

### Next step

```
batch_sub_field1_1 = [
    ["val1", "val3"],
    ["val9", "val11"]
]

batch_sub_field1_2 = [
    ["val2", "val4"],
    ["val10", "val12"]
]

batch_sub_field2_1 = [
    ["val5", "val7"],
    ["val13", "val15"]
]

batch_sub_field2_2 = [
    ["val6", "val8"],
    ["val14", "val16"]
]
```

### Next step (target form)

```
batch_sub_field1_1 = [
    [vec1, vec3],
    [vec9, vec11]
]

batch_sub_field1_2 = [
    [vec2, vec4],
    [vec10, vec12]
]

batch_sub_field2_1 = [
    [vec5, vec7],
    [vec13, vec15]
]

batch_sub_field2_2 = [
    [vec6, vec8],
    [vec14, vec15]
]
```

objects can be nested (in the limited way) so we have to have ways of embedding:
- basic types: strings encoded with text2vec (done with [Universal Sentence Encoder](https://github.com/MartinoMensio/spacy-universal-sentence-encoder))
- lists: Lists need flattening of the temporal dimension (using e.g.: mean() ) - done using so called handlers
- dicts: no problemo, we split them into separate batches - done using so called handlers

Theoretical time complexity is linear :)

### Execution time tests

In [5]:
Service.objects[:1]

[<Service: Service object>]

In [9]:
%timeit Services2tensorsEmbedder()

2.1 s ± 24.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
services2tensors_embedder = Services2tensorsEmbedder()

In [7]:
%timeit services2tensors_embedder(Service.objects[:1])

15.8 ms ± 500 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
x, d = services2tensors_embedder(Service.objects[:10])

In [20]:
import spacy_universal_sentence_encoder

In [21]:
nlp = spacy_universal_sentence_encoder.load_model("en_use_lg")

In [32]:
nlp("I have").doc.

I have

In [23]:
%timeit services2tensors_embedder(Service.objects[:100])

1.12 s ± 222 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit Users2tensorsEmbedder()

308 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
users2tensors_embedder = Users2tensorsEmbedder()

In [19]:
x, d = users2tensors_embedder(User.objects[:10])
x.shape

torch.Size([10, 1024])

In [22]:
%timeit users2tensors_embedder(User.objects[:1])

41.3 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit users2tensors_embedder(User.objects[:100])

97 ms ± 21.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
import pickle

In [39]:
with open("./xd", "wb") as file:
    pickle.dump([1,2,3], file)

In [40]:
with open("./xd", "rb") as file:
    xd = pickle.load(file)
    print(xd)

[1, 2, 3]


## Discussion

Now we can embedd Users, Services or almost any other objects without any embedders training and whole dataset collection in real time. We can do it to new objects that we've never seen before. We are not constrained by possible values of object fields or by old DB dumps or by existing or not existing IDs. Data can be in the any language!!!

It's a little bit too good to be true, so extensive tests of recommendation power needed.

NLP part could and should be vastly extended and improved, for now there is necessary minimum and no more.

Some mechanism for dimensinality reduction should be proposed, e.g.:
- polling in the first layers of networks that deal with user/services
- NLP-based sentences concatenation

If above ideas works (tests needed) the big part of the recommender system that deals with data preparation could be removed.

## Conclusion

> Is it possible to ingest MP DB changes as events in the Recommender and use them for online training and how exactly should it be done?
> -- <cite>[Data Ingestion Redesign Issue #348](https://github.com/cyfronet-fid/recommender-system/issues/348)</cite>

> ### Yes, it's possible and can be done using already implemented technique presented above

What the excellent PR should have:
- implementation
- tests
- refactoring done
- docstrings, examples, readme
- typing
- black, pylint run
- theoretical time and memory complexity
- practical time and memory complexity: profiling
- usage of multiprocesing
- usage of loggers, verbosity
- progress bars if needed